In [1]:
import kfp
import kfp.dsl as dsl
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics,
)

In [3]:
@component
def add(a: float, b: float) -> float:
  '''Calculates sum of two arguments'''
  return a + b

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
  name='addition-pipeline',
  description='An example pipeline that performs addition calculations.',
  pipeline_root='gs://my-pipeline-root/example-pipeline'
)
def add_pipeline(
  a: float=1,
  b: float=7,
):
  # Passes a pipeline parameter and a constant value to the `add` factory
  # function.
  first_add_task = add(a, 4)
  # Passes an output reference from `first_add_task` and a pipeline parameter
  # to the `add` factory function. For operations with a single return
  # value, the output reference can be accessed as `task.output` or
  # `task.outputs['output_name']`.

# Specify pipeline argument values
arguments = {'a': 7, 'b': 8}

In [4]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

project_id = 'c4ds-gcp'
pipeline_root_path = 'gs://c4ds/vertexai/pipelines'

# Define the workflow of the pipeline.
@dsl.pipeline(
    name='addition-pipeline',
    description='An example pipeline that performs addition calculations.',
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str):
    
    first_add_task = add(1, 4)
    second_add_task = add(first_add_task.output, 2)

In [5]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='add_numbers_pipeline.json')

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [42]:
from typing import NamedTuple
@component
def multiple_return_values_example(a: float, b: float) -> NamedTuple(
  'ExampleOutputs',
  [
    ('sum', float),
    ('product', float)
  ]):
    """Example function that demonstrates how to return multiple values."""  
    sum_value = a + b
    product_value = a * b

    from collections import namedtuple
    example_output = namedtuple('ExampleOutputs', ['sum', 'product'])
    return example_output(sum_value, product_value)

In [43]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

project_id = 'c4ds-gcp'
pipeline_root_path = 'gs://c4ds/vertexai/pipelines'

# Define the workflow of the pipeline.
@dsl.pipeline(
    name='multiple-outputs-pipeline',
    description='An example pipeline that performs addition calculations.',
    pipeline_root=pipeline_root_path)

def pipeline(
    project_id: str,
    a: float=4,
    b: float=6
):
    
    first_task = multiple_return_values_example(a, 4)
    second_task = add(first_task.outputs['sum'], b)
    third_task = add(first_task.outputs['product'], b)

In [44]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='3_tasks_pip.json')